In [ ]:
# (C) Copyright IBM Corp. 2019, 2020, 2021, 2022.

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#           http://www.apache.org/licenses/LICENSE-2.0

#     Unless required by applicable law or agreed to in writing, software
#     distributed under the License is distributed on an "AS IS" BASIS,
#     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#     See the License for the specific language governing permissions and
#     limitations under the License.


In [ ]:
import numpy as np
import requests
import rasterio
import zipfile
import os
import datetime
from dateutil.relativedelta import relativedelta
import calendar
import matplotlib.pyplot as plt
from cartopy import config
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point

from simulai.rom import POD
from simulai.models import ModelPool

In [ ]:
def getSSTA():
    url_sst_anomaly="http://iridl.ldeo.columbia.edu/expert/SOURCES/.NOAA/.NCEP/.EMC/.CMB/\
    .GLOBAL/.Reyn_SmithOIv2/.monthly/.ssta/startcolormap/DATA/-5./5./RANGE/transparent/navy/blue/\
    -5./VALUE/cyan/-0.5/VALUE/white/white/0.5/bandmax/yellow/0.5/VALUE/red/5./\
    VALUE/firebrick/endcolormap/DATA/0.5/STEP/data.nc"
    if not os.path.isfile("data.nc"):
        req=requests.get(url_sst_anomaly)
        open("data.nc", "wb").write(req.content)
    return rasterio.open("data.nc")

In [ ]:
print("Executing the data download ...")
ssta = getSSTA()
print("Download is concluded.")

In [ ]:
begin=datetime.datetime.strptime(ssta.tags()['T#units'][-10:], "%Y-%m-%d")
band2date = {j+1 : begin + relativedelta(months=int(float(i))) \
              for j, i in enumerate(ssta.tags()["NETCDF_DIM_T_VALUES"][1:-1].split(","))}
date2band = {j:i for i,j in band2date.items()}

band2date[1], band2date[list(band2date.keys())[-1]]

mask = (1-np.isnan(ssta.read(1)).astype(np.int))

END=2012
years = [i for i in range(1982, END)]

def getBand(i):
    return ssta.read(i)[mask == 1]

def getRow(year):
    return np.vstack([getBand( date2band[datetime.datetime(year, month, 1)] ) for month in range(1,13)])

S = np.vstack([getRow(year) for year in years]).T
S = S.T
n_samples = S.shape[0]

train_S = S[:n_samples - 12]
test_S = S[n_samples - 12:]

Applying POD and projecting the dataset into the ROM basis.

In [ ]:
shuffle = True

rom_config = {
              'n_components': 200,
              'mean_component': True
             }
rom = POD(config=rom_config)
rom.fit(data=train_S)
projected_S = rom.project(data=train_S)

Rescaling data.

In [ ]:
max_proj_S = projected_S.max(0)
min_proj_S = projected_S.min(0)
activation = 'tanh'

if activation != 'sigmoid':
    norm_projected_S = 2*(projected_S - min_proj_S)/(max_proj_S - min_proj_S) - 1
else:
    norm_projected_S = (projected_S - min_proj_S) / (max_proj_S - min_proj_S)

Setups for ESN-RC and parallelism. 

In [ ]:
pool_config = {'template': 'independent_series',
               'n_inputs': 200,
               'n_auxiliary': 0,
               'n_outputs': 200}

rc_config = {
             'reservoir_dim': 5000, 'sparsity_level': 1,
             'radius': 0.99, 'activation': activation,
             'sigma': .1, 'beta': 1e-4,
             'transformation': 'T1'
            }

In [ ]:
input_data = norm_projected_S[:-1]
target_data = norm_projected_S[1:]

initial_state = norm_projected_S[-1:]
horizon = 12

Instantiating the model. 

In [ ]:
pool = ModelPool(config=pool_config, model_type='EchoStateNetwork',
                 model_config=rc_config)

if shuffle:
    indices = np.arange(input_data.shape[0])
    np.random.shuffle(indices)
    input_data_ = input_data[indices, :]
    target_data_ = target_data[indices, :]
else:
    input_data_ = input_data
    target_data_ = target_data

Executing the model fitting.

In [17]:
pool.fit(input_data=input_data_, target_data=target_data_)

KeyboardInterrupt: 

Inverse rescaling transformation. 

In [ ]:
norm_projected_S_tilde = pool.predict(initial_state=initial_state,
                                      horizon=horizon)
if activation != 'sigmoid':
    projected_S_tilde = (max_proj_S - min_proj_S)*(norm_projected_S_tilde + 1)/2 + min_proj_S
else:
    projected_S_tilde = (max_proj_S - min_proj_S) * norm_projected_S_tilde + min_proj_S

Reconstructing the original space. 

In [ ]:
S_tilde = rom.reconstruct(projected_S_tilde)

Post-processing. 

In [ ]:
years = [i for i in range(END,END+1)]

def getBand(i):
    return ssta.read(i)[mask==1]

def getRow(year):
    return np.vstack([getBand( date2band[datetime.datetime(year, month, 1)] ) for month in range(1,13)])

xhat = S_tilde.T
S_true = test_S.T

lats = np.linspace(ssta.bounds.top, ssta.bounds.bottom, ssta.shape[0] + 1)[0:-1]
lons = np.linspace(ssta.bounds.left, ssta.bounds.right, ssta.shape[1] + 1)[0:-1]

plt.figure(figsize=(16, 46))

for i in range(12):
    xtrue = S_true[:, i]  # true value
    xpred = xhat[:, i] #np.real(xhat[S.shape[0]:S.shape[0] * 2, i])
    xtrueview = np.zeros_like(mask)
    xtrueview[mask == 1] = xtrue
    xpredview = np.zeros_like(mask)
    xpredview[mask == 1] = xpred
    # MIN=np.min(np.stack([xhat, xprime]))
    # MAX=np.max(np.stack([xhat, xprime]))
    ax = plt.subplot(12, 2, i * 2 + 1, projection=ccrs.PlateCarree())
    cntr = ax.contourf(lons, lats, xtrueview, levels=np.linspace(-4., 4, 10), transform=ccrs.PlateCarree(), \
                       cmap="bwr", vmin=-4, vmax=4)
    ax.coastlines()
    plt.colorbar(cntr)
    plt.title(f"True anomaly {calendar.month_name[i + 1]} {END}")
    ax = plt.subplot(12, 2, i * 2 + 2, projection=ccrs.PlateCarree())
    cntr = ax.contourf(lons, lats, xpredview, levels=np.linspace(-4., 4, 10), transform=ccrs.PlateCarree(), \
                       cmap="bwr", vmin=-4, vmax=4)
    ax.coastlines()
    plt.colorbar(cntr)
    plt.title(f"Predicted anomaly {calendar.month_name[i + 1]} {END}")

plt.savefig("figure_{}.png".format(activation))

Linear activation

![Linear activation](figures/figure_linear.png)

Hyperbolic tangent activation

![Hyperbolic tangent activation](figures/figure_tanh.png)

Sigmoid activation

![Sigmoid activation](figures/figure_sigmoid.png)